### Langchain e llamacpp
Questo notebook contiene un esempio di utilizzo di langchain invocando un modello "locale" grazie a llamacpp.

E' necessario lanciare il server OPENAI utilizzando le istruzioni presenti nel repo llamacpp.

Grazie a llamacpp è possibile simulare "in locale" un server compatibile con le API OpenAI (vedi http://localhost:8000/docs)

In [1]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )


In [11]:
output = llm.invoke("Spiegami la meccanica quantistica")
print(output.content)

 Spiegami la meccanica quantistica è una richiesta molto ambiziosa che sarebbe necessaria un'intera opera. Tuttavia, io ti porterò in una breve introduzione alla materia.

La meccanica quantistica è una teoria matematica e fisica che descrive il comportamento dei sistemi fisici a microscopico e per questo motivo può essere applicata anche ai macroscopici. Essa fu sviluppata dall'italiano Niels Bohr nella prima metà del XX secolo e poi ulteriormente sviluppata da Werner Heisenberg, Erwin Schrödinger, Paul Dirac e altri fisici.

La meccanica quantistica ha come base due principi fondamentali: la teoria di uncertainty ed il principio di superposizione. La teoria di uncertainty è stato sviluppata da Werner Heisenberg e afferma che non può essere determinato, in una certa misura, sia l'energia che la posizione di un particolare fermione o bosone. Il principio di superposizione afferma che ogni sistema fisico è descritto dalla sua funzione d'onda, che è una funzione matematica che descrive i

In [8]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage)

messages =[
    SystemMessage(content="you are a fhisician and respond only in italian"),
    HumanMessage(content="Spiegami la meccanica quantistica")
]

output = llm.invoke(messages)
print(output.content)

 La meccanica quantistica è una disciplina scientifica che studia i fenomeni fisici a livello atomico e subatomico. Essa deriva dalla teoria delle onde e della relatività, ed è stata sviluppata nel corso del XX secolo per spiegare le proprietà dei subatomi e dell'interazione tra essi.

La meccanica quantistica introduce la concezione di particelle minime indivisibili, che possono essere in più stati allo stesso tempo (superposizione) e si possono trasformare l'una nell'altra (interazione). Questa teoria descrive anche il comportamento dei fotoni (quanti di energia), che sono i fotoni e le particelle elementari antiprotone e protone.

La meccanica quantistica è utilizzata in molte aree di ricerca, tra cui la fisica delle particelle e la cosmologia.


### Streaming
Per gestire la risposta con lo streaminig

In [9]:
chunks = []
async for chunk in llm.astream("hello. tell me something about yourself"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

 Sure! I am a language model trained by Mistral AI, a cutting-edge company from France. My goal is to help people find information and answer their questions in the best possible way. I have access to a vast amount of data and can understand natural language, so feel free to ask me anything you like!

### Caching

In memory cache

In [15]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )

In [16]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 72.5 ms, sys: 4.01 ms, total: 76.5 ms
Wall time: 11.6 s


'. Why did the tomato turn red? Because it saw the salad dressing!'

In [17]:
%%time
llm.invoke(prompt)

CPU times: user 1.11 ms, sys: 274 µs, total: 1.39 ms
Wall time: 1.42 ms


'. Why did the tomato turn red? Because it saw the salad dressing!'

In [24]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 14.3 ms, sys: 1.7 ms, total: 16 ms
Wall time: 13.4 ms


'. Why did the chicken cross the road? To get to the other side! Why did the tomato turn red? Because it saw the salad dressing!'

In [25]:
%%time
llm.invoke(prompt)

CPU times: user 7.12 ms, sys: 0 ns, total: 7.12 ms
Wall time: 5.53 ms


'. Why did the chicken cross the road? To get to the other side! Why did the tomato turn red? Because it saw the salad dressing!'

### Streaming

In [27]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )

In [28]:
prompt = "Racconta una storia sulla luna e le stelle"
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

 Once upon a time, there was a little girl named Lily who loved nothing more than to gaze up at the night sky and see all the twinkling stars. She would often lay on her back in the grass and stare for hours, lost in wonder at how vast and mysterious the universe was.

One night, as she was looking up at the stars, she noticed a bright light shining in the sky. It was different from any of the other stars she had seen before, and it seemed to be getting brighter and bigger by the minute.

Lily was so fascinated by this light that she decided to do some research. She learned that the light was coming from the moon, which is Earth's only natural satellite. The moon was so close to Earth that it appeared to be much larger than it really was.

As Lily continued to stare up at the moon, she noticed something strange happening. It seemed as though the moon was changing shape, from a crescent to a quarter to a gibbous and finally to a full moon.

Lily was amazed by this phenomenon and decided

### PromptTemplates

In [32]:
from langchain.prompts import PromptTemplate 
template = '''You are an expert vitologist. Write a few sentences about that virus {virus} in {language}'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='italian')

In [33]:
prompt

'You are an expert vitologist. Write a few sentences about that virus hiv in italian'

In [35]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    
    )
output = llm.invoke(prompt)
print(output.content)

 Il virus HIV, o Human Immunodeficiency Virus, è un virus che attacca e distrugge le cellule dell'immunosistema umano, causando la malattia dell'AIDS. È trasmesso principalmente attraverso il contatto sessuale non protetto, ma anche attraverso l'inoculazione di sangue infettato o attraverso la nascita da madre a figlio. Il virus è in grado di mutare rapidamente, rendendo difficile la creazione di farmaci efficaci per il trattamento.


### ChatPromptTemplate

In [41]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = 'You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} cities in  {country} by population order by population descending.')
    ]
)

messages = chat_template.format_messages(n = '10', country = 'Italy')
print(messages)

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 10 cities in  Italy by population order by population descending.')]


In [42]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    
    )
output = llm.invoke(messages)
print(output.content)

 {
"cities": [
{
"name": "Milan",
"population": 2693847
},
{
"name": "Rome",
"population": 2833020
},
{
"name": "Naples",
"population": 2187553
},
{
"name": "Florence",
"population": 1946847
},
{
"name": "Torino",
"population": 1904423
},
{
"name": "Venice",
"population": 1509189
},
{
"name": "Genoa",
"population": 1567187
},
{
"name": "Bologna",
"population": 1087244
},
{
"name": "Pisa",
"population": 97734
},
{
"name": "Reggio Emilia",
"population": 92657
}
]
}
